# Chapter 6 – Programming Paradigms
## Introduction
Programming languages can be classified into one or more *paradigms*. Knowing the paradigm gives the programmer a hint towards what sort of things the language is designed to do well, and how they should structure their code.

When we first started writing code back in the first chapter we just wrote command after command; each line to be executed in turn from the top to the bottom. Programmers often have use for short simple snippets of code like this. Maybe I have a bunch of images on my desktop that I want to resize in a specific way... it might be easier to write a short Python program than find some batch tool that does exactly what I want. We might call this a **script**, a simple program that just does one thing with limited options and has no real structure.

Over time if we keep tweaking that script and adding new features we are eventually going to end up with bloated, messy, hard to read code. We might wish that we had written the code from scratch with some actual structure. This is why we introduced *functions* so early in the unit. The exact way we structure the code of a large program depends on what paradigm we are using.

Python is an interesting language because it supports so many different programming paradigms. We are going to learn about *object-oriented programming*, which is the most popular paradigm today, and we will also briefly mention some concepts from *functional programming*. But first let's formalise what we've been doing in the unit so far.

## 6.1 – Procedural Programming
Hopefully the Python code that you have been writing outside of Jupyter looks something like this:
```python
# define some functions
def function1(...):
    # ...
    
def function2(...):
    # ...
    
def function3(...):
    # ...
    
# main code goes here – i.e. use the functions
x = function1(...)
```

This is an example of the **procedural programming** paradigm. The solution to the task is broken down with variables, data structures, and crucially: procedures (and functions!). This is the programming paradigm that is favoured by languages like C.

In C, all code must be written inside a subroutine, but there is a special subroutine called `main`. If a file called `program.c` is compiled and run, then the `main` subroutine in that file is the code which is run first. In Python we can write this code “outside” of our subroutine definitions, as we do in the example above (the final line of the example). But there is a slight issue with doing this in real programs.

If we are writing lots of useful functions and procedures, we might want to allow other users to *import* our code as a module (see [Section 5.1](../Chapter%205/5.1.ipynb)). If so, we do not necessarily want the “outside” code to run – it is probably test code to make sure our functions actually work, not something other programs want to happen automatically. Ideally, we would like the code “outside the subroutines” to run only when the file itself is run as a Python program, not when it is imported into another file. 

We can check the current *namespace* to achieve this. It is just a one line modification to the code above, and gives us a structure that ends up looking quite similar to a C `main` subroutine:
```python
# define some functions
def function1(...):
    # ...
    
def function2(...):
    # ...
    
def function3(...):
    # ...
    
if __name__ == "__main__":
    # main code goes here – i.e. use the functions
    x = function1(...)
```

This line of code, `if __name__ == "__main__":` is common to see in Python programs which are intended to be run, even if there is no intention for the code to ever be imported by someone else.

### Limitations
Procedural programming was the dominant paradigm for a long time, but some of its behaviours can end up being quite inelegant. This is particularly prominent when we try to implement our own data structures.

#### Stack Data Structure
A **stack** is a collection of items, to which we can only add one item at a time, and we can only retrieve one item at a time. The order is *last in, first out*. So you can think of a stack as being like a stack of plates: the last item you add to the top of the stack will be the first one you retrieve. Adding an item to a stack is called *pushing* and removing the top item of a stack is called *popping*. 

Now, Python is a language of convenience! Its builtin list type already supports `append` and `pop`. But those features must have been coded by *someone*, and educationally it is interesting to try to work out if we can recreate them. 

So, what if we wanted to create our own stack data type based on Python lists, *where we are only allowed to use positive list indexing*? So `list[0]` is allowed, but `list.pop()` is not allowed. We are simulating the use of a traditional *array* with a Python *list* – and an array is what Python is actually using *under the hood* to make its list type work in the first place.

A stack can be implemented with an array in the following way:
* To create a new stack of size `size`:
 * Create an array `contents` with room for `size` elements
 * Create a variable called `head` set to `0`, which *points* at the next available space
* To push an item `x`:
 * Check if the stack is full, if it is, give an error, otherwise continue
 * Add `x` to `contents` at position `head`
 * Increase `head` by one
* To pop an item:
 * Check if the stack is empty, if it is, give an error, otherwise continue
 * Decrease `head` by one
 * Return the element at position `head` of `contents`
 
Below is an animation demonstrating the following commands in order (you might need to click play on the video):
* New stack, size 5
* Push 4
* Push 10
* Push 3
* Pop
* Pop
* Push 7
* Pop

<br /><video controls loop autoplay width=240 src="./resources/stack.mp4">
</video>

#### Stack Implementation
Let's create a procedural Python implementation of a stack.

In [13]:
def new_stack(size):
    # our stack will be a list of [list, int] for [contents, head]
    # so stack[0] is contents
    #    stack[1] is head
    contents = [0] * size
    head = 0
    return [contents, head]

def is_full(stack):
    return stack[1] > len(stack[0])
    
def is_empty(stack):
    return stack[1] == 0    

def push(stack, x):
    if is_full(stack):
        # fail silently, could also raise an error here instead
        return
    stack[0][stack[1]] = x
    stack[1] += 1
    
def pop(stack):
    if is_empty(stack):
        return
    stack[1] -= 1
    return stack[0][stack[1]]

if __name__ == "__main__":
    my_stack = new_stack(5)
    push(my_stack, 4)
    push(my_stack, 10)
    push(my_stack, 3)
    print(pop(my_stack)) # should print 3
    print(pop(my_stack)) # should print 10
    push(my_stack, 7)
    print(pop(my_stack)) # should print 7

3
10
7


This is, more or less, how people used to implement their own data structures in a procedural language. And it works! If we saved the code above as `stack.py` then we could use these stack functions in another program by writing `import stack`, like this:
```python
import stack

my_stack = stack.new_stack(5)
stack.push(my_stack, 4)
```

The code at the bottom is protected by the “if name equals main” line, so won't be run when the code is imported. Perfect for test code like this.

Spend some time looking at the code above. Modify the code so to push additional elements onto the stack which go beyond its size, then seeing what happens when you pop an element.

#### However

This isn't really the preferred method in Python. The code is a bit inelegant, in a few ways.

First is not the fault of procedural programming. I chose to store the stack as a list of two items because then we could pass that list around and modify its elements in different procedures. But this leads to awkward lines like `stack[0][stack[1]] = x`. 

In a “proper” procedural language you can normally create *structures* that make this syntax a bit nicer by allowing for a variable with named elements. For example we might be able to make a structure which always contains an array called `contents` and an integer called `head`, so then we could do: `stack.contents[stack.head] = x`, much nicer.

We actually can simulate this in Python too, but to do so we need to use objects (which are coming in the next section, so we'll come back to this).

So we'll ignore that slightly awkward syntax for now. But there are other problems more fundamental to procedural programming. One is that the user of this stack library still has direct access to the underlying data. In the example below we access the bottom element of the stack by accessing the underlying contents array. Even if we used a structure this would be unavoidable. We haven't really met the brief of making a data structure that can *only* be accessed first-in-last-out.

In [11]:
# access the bottom element of the stack directly
my_stack[0][0]

4

Remember in the unit introduction we said that we like to distinguish between *data* and *actions*. The `my_stack` variable is stack in terms of its data only. The variable itself has no notion of the possible *actions* available to a stack.

That brings us back to those inbuilt *push* and *pop*-like features of Python lists. With a list, we can do this:

In [12]:
my_stack = []
my_stack.append(4)
my_stack.append(10)
my_stack.append(3)
print(my_stack.pop()) # should print 3
print(my_stack.pop()) # should print 10
my_stack.append(7)
print(my_stack.pop()) # should print 7

3
10
7


The `append` and `pop` *methods* belong to the list variable itself. This syntax, `variable.method()` is possible because Python lists are actually ***objects***, a word we've used before, but soon will give a precise meaning to.

And that takes us right into the subject of [the next section](6.2.ipynb).